In [13]:
import sys
sys.path.insert(1, "/home/p11333at/nlp-project/script/")
from openai_sentiment import OpenAISentiment
import os
import re
import json
import numpy as np
import spampy


filename = "tweets_test_#100.jsonl"
raw_path = '../data/raw/'
processed_path = '../data/processed/'

## Filter out irrelevant tweets

## Random Sampling


In [2]:
files = sorted(os.listdir(raw_path))

dataset_size = []
filepath_list = []
for filename in files:
    filepath = os.path.join(raw_path, filename)
    filepath_list.append(filepath)
    with open(filepath) as f:
        data = f.readlines()
        dataset_size.append(len(data))

dataset_size = np.array(dataset_size)

sample_size = dataset_size // 1000

print(f'Dataset size: {dataset_size}')
print(f'Sample size: {sample_size}')
print(f'The total number of sample: {sample_size.sum()}')

### If you want to skip the some files, use the lines below
# filepath_list = filepath_list[10:]
# dataset_size = dataset_size[10:]
# sample_size = sample_size[10:]
filepath_list, dataset_size, sample_size

Dataset size: [63481 70991 68084 60474 54884 47763 52763 56311 54374 53808 55898 53839
 53074 48168 48605 49871]
Sample size: [63 70 68 60 54 47 52 56 54 53 55 53 53 48 48 49]
The total number of sample: 883


(['../data/raw/tweets_230227_#63481.json',
  '../data/raw/tweets_230228_#70991.json',
  '../data/raw/tweets_230301_#68084.json',
  '../data/raw/tweets_230302_#60474.json',
  '../data/raw/tweets_230303_#54884.json',
  '../data/raw/tweets_230304_#47763.json',
  '../data/raw/tweets_230305_#52763.json',
  '../data/raw/tweets_230306_#56311.json',
  '../data/raw/tweets_230307_#54374.json',
  '../data/raw/tweets_230308_#53808.json',
  '../data/raw/tweets_230309_#55898.json',
  '../data/raw/tweets_230310_#53839.json',
  '../data/raw/tweets_230311_#53074.json',
  '../data/raw/tweets_230312_#48168.json',
  '../data/raw/tweets_230313_#48605.json',
  '../data/raw/tweets_230314_#49871.json'],
 array([63481, 70991, 68084, 60474, 54884, 47763, 52763, 56311, 54374,
        53808, 55898, 53839, 53074, 48168, 48605, 49871]),
 array([63, 70, 68, 60, 54, 47, 52, 56, 54, 53, 55, 53, 53, 48, 48, 49]))

In [3]:
# indexes = []
# for dataset, sample in zip(dataset_size, sample_size):
#     np.random.seed(42)
#     indexes.append(np.random.choice(dataset, size=sample, replace=False))
# indexes = np.array(indexes, dtype=object)

In [16]:
sample_output = []
for i, filepath in enumerate(filepath_list):
    with open(filepath, 'r') as f:
        # Read as json
        data = f.readlines()
        data = [json.loads(line) for line in data]

        # Get the sample indexes
        d_size = dataset_size[i]
        s_size = sample_size[i]
        np.random.seed(42)  # Set seed for reproducibility
        index = np.random.choice(d_size, size=s_size, replace=False)
        index = reversed(sorted(index)) # Sort in descending order

        # Append the sample to the output
        sample_output.extend([data[i] for i in index])


In [3]:
# Retrieve data to increase positive samples for balanced dataset

# sample_output = []
# for i, filepath in enumerate(filepath_list):
#     with open(filepath, 'r') as f:
#         # Read as json
#         data = f.readlines()
#         data = [json.loads(line) for line in data]

#         # Get the sample indexes
#         d_size = dataset_size[i]
#         s_size = sample_size[i]
#         np.random.seed(42)  # Set seed for reproducibility
#         index = np.random.choice(d_size, size=s_size*3, replace=False)[s_size*2:]
#         index = reversed(sorted(index)) # Sort in descending order

#         # Append the sample to the output
#         sample_output.extend([data[i] for i in index])


In [4]:
# Retrieve the date range
file_dates = list(map(lambda x: re.search(r"(?<=\_)\d{6}(?=\_)", x).group(), filepath_list))
start_date, end_date = file_dates[0], file_dates[-1]

print(f'Date range: {start_date} - {end_date}')

Date range: 230227 - 230314


In [5]:
# Set the path and filename
path = "/home/p11333at/nlp-project/data/samples/"
filename = f"samples_{start_date}-{end_date}_#{len(sample_output)}.json"
# filename = f"positive_2_{start_date}-{end_date}_#{len(sample_output)}.json"

# Write the sample to file
with open(f"{path}{filename}", "w") as f:
    for line in sample_output:
        json.dump(line, f)
        f.write('\n')

if os.path.exists(f"{path}{filename}"):
    print(f"The file '{filename}' is created.")

The file 'positive_2_230227-230314_#883.json' is created.


## Spam detection

In [39]:
test = "USERNAME_01 USERNAME_02 USERNAME_03 Some months back I had no way of taking care of my kids , had no job or money also struggling through the pandemic,so I decided to try Bitcoin trading with the help of USERNAME_04 I was able to earn R130,000 now am financially stable."
def detect_spam(tweet):
    content = tweet['content']
    bitcoin_spam = [r"Some months back I had no way of taking care of my kids", r"had no job or money", r"try Bitcoin trading"]
    pattern_spam = [bitcoin_spam]
    for patterns in pattern_spam:
        for p in patterns:
            if not re.search(p, content):
                break
        else:
            return True
    return False

# detect_spam(test)
fname = "../data/raw/tweets_230227_#63481.json"

In [40]:
with open(fname, 'r') as f:
    data = f.readlines()
    data = [json.loads(line) for line in data]

cleaned_data = [line for line in data if not detect_spam(line)]


In [ ]:
with open(fname, 'w') as f:
    for line in cleaned_data:
        json.dump(line, f)
        f.write('\n')

In [38]:
spam = filter(detect_spam, cleaned_data)
spam = list(spam)
spam

[]